In [ ]:
################# script area: 1
#############################
import pickle
with open('output/dialo/dialo_train', 'rb') as f:
    data = pickle.load(f)

In [ ]:
################ script area: 2
i = 0
for dialo in data[i:i+1]:
    src, lbl, tkn_typ = dialo
    

In [ ]:
################# 生成测试
#########################

import os
import torch
from transformers import BertTokenizer, BertForMaskedLM
from transformers import logging

os.chdir('E:/Local-Data/Enconter/')
logging.set_verbosity_error()
MODEL = 'output/dialo_ckpt/model_9.ckpt'

tokenizer = BertTokenizer.from_pretrained('output/dialo_ckpt/bert-base-chinese/')

model = BertForMaskedLM.from_pretrained('../models_datasets/bert-base-chinese/')
model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(torch.load(MODEL, map_location='cpu'))

model.eval()

In [ ]:
################# 测试
pat = '[CLS][PAT]我是肚子疼'
doc = '[DOC]胃病[SEP]'
pat_ids = tokenizer.encode(pat, add_special_tokens=False)
doc_ids = tokenizer.encode(doc, add_special_tokens=False)

input_ids = pat_ids + doc_ids
span = range(len(pat_ids), len(input_ids))

input_ids = input_ids + [tokenizer.pad_token_id]*(512-len(input_ids))
token_type_ids = [0]*len(pat_ids)+[1]*(512-len(pat_ids))

input_ids = torch.tensor([input_ids], dtype=torch.long)
token_type_ids = torch.tensor([token_type_ids], dtype=torch.long)
position_ids = torch.arange(512, dtype=torch.long).unsqueeze(0)

print(f'input_ids: {input_ids.shape}, token_type_ids: {token_type_ids.shape}, position_ids: {position_ids.shape}')

logits = model(input_ids=input_ids, token_type_ids=token_type_ids, position_ids=position_ids)['logits']
output = torch.argmax(logits, dim=-1).flatten().tolist()

# new_output = []
# for doc_id, o in zip(doc_ids, output):
#     new_output.append(doc_id)
#     new_output.append(o)

print(tokenizer.decode(output))